# Curl

In [8]:
# 안되면 1-3주차 13분30초
!curl http://httpbin.org/get?myname="js"

zsh:1: no matches found: http://httpbin.org/get?myname=js


# Description #


In [6]:
import requests

In [7]:
r = requests.options("http://httpbin.org/")

In [8]:
print(r.headers['allow'])

OPTIONS, HEAD, GET


## HTTP 의 코드번호 상태 ##
- 200: 앞자리 2는 성공을 의미, 200은 성공적으로 처리되었다는 뜻
- 300: 앞자리 3은 우회를 의미, 300은 클라이언트가 선택해야 한다는 뜻
- 404: 앞자리 4는 클라 오류, 404는 Not Found, 403은 Forbidden
- 500: 앞자리 5는 서버 오류, 500은 서버 오류, 501은 서버 미구현

In [9]:
r.status_code

200

In [10]:
r = requests.post("http://httpbin.org/post", data = {"name" : "js"})
r.json()

{'args': {},
 'data': '',
 'files': {},
 'form': {'name': 'js'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '7',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.26.0',
  'X-Amzn-Trace-Id': 'Root=1-6145afa6-124308723e845e3e0806922a'},
 'json': None,
 'origin': '220.89.202.18',
 'url': 'http://httpbin.org/post'}

In [11]:
r = requests.head("http://httpbin.org/")
print(r.headers)
print(r.headers['Content-Type'])

{'Date': 'Sat, 18 Sep 2021 09:21:43 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '9593', 'Connection': 'keep-alive', 'Server': 'gunicorn/19.9.0', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true'}
text/html; charset=utf-8


# urllib

In [12]:
import urllib

class HeadRequest(urllib.request.Request):
    def get_method(self):
        return "HEAD"

response = urllib.request.urlopen(HeadRequest("http://httpbin.org/"))

In [13]:
print(response.info())

Date: Sat, 18 Sep 2021 09:21:45 GMT
Content-Type: text/html; charset=utf-8
Content-Length: 9593
Connection: close
Server: gunicorn/19.9.0
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true




In [14]:
response.geturl()

'http://httpbin.org/'

# 많이 쓰이는 웹데이터 형식
- XML
- JSON

## XML
> Extensible Markup Language

In [15]:
input = '''
<students>
    <student x="1">
        <id>001</id>
        <name>Kim</name>
    </student>
    <student x="2">
        <id>002</id>
        <name>Lee</name>
    </student>
</students>
'''

In [17]:
import lxml.etree
root = lxml.etree.fromstring(input) # 문자열에서 읽는다.

In [19]:
from io import StringIO
tree = lxml.etree.parse(StringIO(input)) # 파일에서 읽는데, StringIO로 파일처럼 읽듯이 한다.
root = tree.getroot()

In [21]:
%%writefile ds_open_hello.xml
<students>
    <student x="1">
        <id>001</id>
        <name>Kim</name>
    </student>
    <student x="2">
        <id>002</id>
        <name>Lee</name>
    </student>
</students>

Writing ds_open_hello.xml


In [25]:
import os
tree = lxml.etree.parse('ds_open_hello.xml')
root = tree.getroot()

In [30]:
print(type(root.tag))
for e in root:
    print(e.tag)

<class 'str'>
student
student


In [33]:
print(root.getchildren())
print(type(root.getchildren()))
print(root.getchildren()[1])
for ee in root.getchildren():
    for e in ee.getchildren():
        if not e.text:
            text = "None"
        else:
            text = e.text
        print(e.tag + " => " + text)

[<Element student at 0x7fefe318a9c0>, <Element student at 0x7fefe3186ec0>]
<class 'list'>
<Element student at 0x7fefe3186ec0>
id => 001
name => Kim
id => 002
name => Lee


In [37]:
for node in root.getiterator():
    print('Tag: {0} \tATTRIB: {1} \t\ttext: {2}'.format(node.tag, node.attrib, node.text))


Tag: students 	ATTRIB: {} 		text: 
    
Tag: student 	ATTRIB: {'x': '1'} 		text: 
        
Tag: id 	ATTRIB: {} 		text: 001
Tag: name 	ATTRIB: {} 		text: Kim
Tag: student 	ATTRIB: {'x': '2'} 		text: 
        
Tag: id 	ATTRIB: {} 		text: 002
Tag: name 	ATTRIB: {} 		text: Lee


## XML 검색
- iterfind() : 반복적으로 인자와 일치하는 요소를 검색
- findall() : 인자와 일치하는 요소들을 리스트로 반환
- find() : 인자와 일치하는 첫 요소를 반환
- findtext() : 인자와 일치하는 첫 요소의 .text 내용을 반환

In [40]:
std = root.find('student')
for node in std:
    print(node.text)
print(std.attrib)

001
Kim
{'x': '1'}


In [42]:
stds = root.findall('student')
for node in stds:
    for item in node:
        print(item.text)

001
Kim
002
Lee


## XPath
- /: 루트 바로 아래 수준만 검색
- //: 모든 부분에서 검색
- @: 속성을 검색

In [43]:
root.xpath('//@x')

['1', '2']

In [45]:
root.xpath('//*[@x="1"]/id/text()')

['001']

## CSS 검색 ##
- . 을 사용해 클래스
- \# 을 사용해 아이디

In [46]:
from lxml.cssselect import CSSSelector
sel = CSSSelector('student')

In [47]:
print(sel)
print(sel.css)
print(sel.path)

<CSSSelector 7fefe31944a0 for 'student'>
student
descendant-or-self::student


In [52]:
root = lxml.etree.fromstring(input)
nodes = sel(root)
print(type(nodes))
print(len(nodes))
for e in nodes:
    print(e.get('x'))

<class 'list'>
2
1
2


In [54]:
sel = CSSSelector('id')
nodes = sel(root)
print(type(nodes))
for e in nodes:
    print(e.text)

<class 'list'>
001
002


JSON
> JavaScript Object Notation

In [57]:
input = '''[
    { "id" : "001", "x" : "2", "name" : "Chuck"},
    { "id" : "009", "x" : "7", "name" : "Brent"}
]'''

In [58]:
import json
info = json.loads(input)

In [59]:
'User count:{}'.format(len(info))

'User count:2'

In [60]:
for item in info:
    print("id: {} \tname: {}".format(item['id'], item['name']))

id: 001 	name: Chuck
id: 009 	name: Brent


In [63]:
import json
my = json.dumps(
    ['foo', {'bar': ('baz', None, 1.0, 2)}]
)
print(type(my))
# 자바스크립트에서 사용하는 문법으로 변환해서 출력할 수 있다.
print(my)

<class 'str'>
["foo", {"bar": ["baz", null, 1.0, 2]}]
